# Flight Status - Final Project

In [65]:
import os
import shutil
import warnings

import pandas as pd
#import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import opendatasets as od

from sqlalchemy import create_engine
from dotenv import load_dotenv


warnings.filterwarnings('ignore')

## TAS-12 Data Acquisition

### Data Acquisition - Use Kaggle API

[How to use Kaggle API - Step-by-step guide](https://www.geeksforgeeks.org/how-to-download-kaggle-datasets-into-jupyter-notebook/)

Since almost all combined CSV files are larger than 60k records and more than 20 columns the best course of action would be to only use one year data, in this case we are going to use the `Combined_Flights_2022.csv` with the shape `(4078318, 61)` and the `Airlines.csv` for labeling if needed

The **Data Acquisition - Use Kaggle API** section can be re-run only in case we need to access the data from the kaggle API again. After this section we cleaned the data and saved it as CSV in the `/data/processed/` folder, accessed from there, then we'll create a DB with 2 tables `airlines` and `flights`, with the corresponding data and we'll work the data by retrieving it from the DB tables

In [66]:
# ONLY RUN ONCE
od.download('https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading flight-delay-dataset-20182022.zip to .\flight-delay-dataset-20182022


100%|██████████| 3.73G/3.73G [01:49<00:00, 36.6MB/s]


In [67]:
# ONLY RUN ONCE
def move_folder():
    source = './flight-delay-dataset-20182022'
    destination = '../data/raw/flight-delay-dataset-20182022'

    shutil.move(source, destination)

In [68]:
# ONLY RUN ONCE
def remove_unnecesary():
    destination = '../data/raw/flight-delay-dataset-20182022'
    try:
        shutil.rmtree(os.path.join(destination, 'raw'))
    except:
        print("Continue")
    finally:
        files = [x for x in os.listdir(destination) if not ('2022.csv' in x or 'Airlines' in x)]
        for f in files:
            os.remove(os.path.join(destination, f))


In [69]:
# ONLY RUN ONCE
move_folder()

In [75]:
# ONLY RUN ONCE
remove_unnecesary()

Continue


PermissionError: [WinError 5] Acceso denegado: '../data/raw/flight-delay-dataset-20182022\\flight-delay-dataset-20182022'

In [76]:
# ONLY RUN ONCE
for file in os.listdir('../data/raw/flight-delay-dataset-20182022'):
    shutil.move(os.path.join('../data/raw/flight-delay-dataset-20182022/', file), os.path.join('../data/processed'), file)

### Data Acquisition - Clean and Store Data to CSV

In [90]:
data = pd.read_csv('../data/processed/Combined_Flights_2022.csv')

In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4078318 entries, 0 to 4078317
Data columns (total 34 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   Unnamed: 0                               int64  
 1   FlightDate                               object 
 2   Airline                                  object 
 3   Cancelled                                bool   
 4   Diverted                                 bool   
 5   DepTime                                  object 
 6   DepDelayMinutes                          float64
 7   DepDelay                                 float64
 8   ArrTime                                  object 
 9   ArrDelayMinutes                          float64
 10  AirTime                                  float64
 11  CRSElapsedTime                           float64
 12  Distance                                 float64
 13  Year                                     int64  
 14  Quarter           

In [92]:
data.describe()

,Unnamed: 0,DepDelayMinutes,DepDelay,ArrDelayMinutes,AirTime,CRSElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,Flight_Number_Operating_Airline,OriginAirportID,DestAirportID,ArrDelay,DivAirportLandings
count,4.078318e+06,3.957823e+06,3.957823e+06,3.944916e+06,3.944916e+06,4.078318e+06,4.078318e+06,4078318.0,4.078318e+06,4.078318e+06,4.078318e+06,4.078318e+06,4.078318e+06,4.078318e+06,4.078318e+06,4.078318e+06,4.078318e+06,4.078318e+06,3.944916e+06,4.078318e+06
mean,2.039158e+06,1.601494e+01,1.309049e+01,1.578307e+01,1.110075e+02,1.413211e+02,7.978657e+02,2022.0,1.741207e+00,4.083803e+00,1.571131e+01,4.005566e+00,1.983101e+04,2.562115e+03,1.999122e+04,2.562145e+03,1.265994e+04,1.265990e+04,7.528486e+00,3.685098e-03
std,1.177309e+06,5.231498e+01,5.332016e+01,5.198424e+01,6.996246e+01,7.179635e+01,5.914742e+02,0.0,7.037563e-01,1.998918e+00,8.760122e+00,2.007050e+00,2.760907e+02,1.745826e+03,3.767694e+02,1.745873e+03,1.522713e+03,1.522718e+03,5.524625e+01,1.141331e-01
min,0.000000e+00,0.000000e+00,-7.800000e+01,0.000000e+00,8.000000e+00,-4.800000e+01,3.100000e+01,2022.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.939300e+04,1.000000e+00,1.939300e+04,1.000000e+00,1.013500e+04,1.013500e+04,-1.000000e+02,0.000000e+00
25%,1.019579e+06,0.000000e+00,-5.000000e+00,0.000000e+00,6.000000e+01,8.900000e+01,3.680000e+02,2022.0,1.000000e+00,2.000000e+00,8.000000e+00,2.000000e+00,1.979000e+04,1.105000e+03,1.979000e+04,1.105000e+03,1.129200e+04,1.129200e+04,-1.400000e+01,0.000000e+00
50%,2.039158e+06,0.000000e+00,-2.000000e+00,0.000000e+00,9.400000e+01,1.240000e+02,6.430000e+02,2022.0,2.000000e+00,4.000000e+00,1.600000e+01,4.000000e+00,1.980500e+04,2.228000e+03,1.997700e+04,2.228000e+03,1.288900e+04,1.288900e+04,-5.000000e+00,0.000000e+00
75%,3.058738e+06,1.100000e+01,1.100000e+01,1.000000e+01,1.410000e+02,1.710000e+02,1.035000e+03,2022.0,2.000000e+00,6.000000e+00,2.300000e+01,6.000000e+00,1.997700e+04,3.877000e+03,2.037800e+04,3.877000e+03,1.402700e+04,1.402700e+04,1.000000e+01,0.000000e+00
max,4.078317e+06,7.223000e+03,7.223000e+03,7.232000e+03,7.270000e+02,6.900000e+02,5.095000e+03,2022.0,3.000000e+00,7.000000e+00,3.100000e+01,7.000000e+00,2.043600e+04,9.680000e+03,2.050000e+04,9.680000e+03,1.686900e+04,1.686900e+04,7.232000e+03,9.000000e+00


In [93]:
def check_nulls(df):
    # Assuming df is your DataFrame
    # Check for null values in each column
    percentage = 10
    percent = (percentage * len(df)) / 100
    null_counts = df.isnull().sum()

    # Filter columns with null values and print their sum
    columns_with_nulls_ten = null_counts[null_counts > percent]
    columns_with_nulls = null_counts[null_counts > 0]
    if len(columns_with_nulls_ten) > 0:
        for column, count in columns_with_nulls.items():
            print(f"Column '{column}' has {count} null values.")
    else:
        print("The null values in the dataframe don't exceed {percent} values or {percentage}% of the total data".format(percent=percent, percentage=percentage))
        print("Depending on Duplicated values we might want to consider dropping them since that low percentage of null values would hardly make any difference in the EDA or the model creation and prediction")

In [94]:
check_nulls(data)

The null values in the dataframe don't exceed 407831.8 values or 10% of the total data
Depending on Duplicated values we might want to consider dropping them since that low percentage of null values would hardly make any difference in the EDA or the model creation and prediction


In [95]:
# data.duplicated().sum()

In [96]:
def treat_nulls(value):
    if pd.isnull(value):
        return 0
    else:
        return int(value)
    
# Define a function to transform values
def transform_time(value):
    if isinstance(value, int):
        value = str(value)  # Convert integer to string
    value = value.zfill(4)  # Pad with leading zeros if necessary
    if len(value) == 4:
        if int(value) == 2400:
            return '00:00'
        if int(value) < 10:  # For values less than 10
            return f'00:0{value[0]}'
        elif int(value) < 100:  # For values between 10 and 100
            return f'00:{value[:2]}'
        else:  # For values over 100
            return f'{value[:2]}:{value[2:]}'
    else:
        return value[:2] + ':' + value[2:]  # Format as 'HH:MM'

In [97]:
def format_dates():
    data['FlightDate'] = pd.to_datetime(data['FlightDate'])

    # Format time WheelsOff
    data['WheelsOff'] = data['WheelsOff'].apply(treat_nulls)
    data['WheelsOff'] = data['WheelsOff'].apply(transform_time)
    data['WheelsOff'] = pd.to_datetime(data['WheelsOff'], format='%H:%M').dt.time

    # Format time WheelsOn
    data['WheelsOn'] = data['WheelsOn'].apply(treat_nulls)
    data['WheelsOn'] = data['WheelsOn'].apply(transform_time)
    data['WheelsOn'] = pd.to_datetime(data['WheelsOn'], format='%H:%M').dt.time

    # Format time ArrTime
    data['ArrTime'] = data['ArrTime'].apply(treat_nulls)
    data['ArrTime'] = data['ArrTime'].apply(transform_time)
    data['ArrTime'] = pd.to_datetime(data['ArrTime'], format='%H:%M').dt.time


    # Format time DepTime
    data['DepTime'] = data['DepTime'].apply(treat_nulls)
    data['DepTime'] = data['DepTime'].apply(transform_time)
    data['DepTime'] = pd.to_datetime(data['DepTime'], format='%H:%M').dt.time

In [98]:
format_dates()

ValueError: invalid literal for int() with base 10: '11:40:00'

In [88]:
origins = data[['OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac']]
origins.drop_duplicates(inplace=True)
origins.to_csv('../data/processed/origins.csv', index = True)

KeyError: "['OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac'] not in index"

In [89]:
destinations = data[['DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac']]
destinations.drop_duplicates(inplace=True)
destinations.to_csv('../data/processed/destinations.csv', index = True)

KeyError: "['DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac'] not in index"

In [ ]:
data.drop(columns=['CRSDepTime','ActualElapsedTime','CRSArrTime', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'TaxiIn', 'TaxiOut', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'DistanceGroup', 'CRSDepTime', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'CRSArrTime', 'CRSElapsedTime'], inplace=True)

In [ ]:
colms = data.columns

print(colms)

Index(['FlightDate', 'Airline', 'Cancelled', 'Diverted', 'DepTime',
       'DepDelayMinutes', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime',
       'CRSElapsedTime', 'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth',
       'DayOfWeek', 'Marketing_Airline_Network',
       'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline',
       'IATA_Code_Marketing_Airline', 'Flight_Number_Marketing_Airline',
       'Operating_Airline', 'DOT_ID_Operating_Airline',
       'IATA_Code_Operating_Airline', 'Tail_Number',
       'Flight_Number_Operating_Airline', 'OriginAirportID', 'DestAirportID',
       'WheelsOff', 'WheelsOn', 'ArrDelay', 'DivAirportLandings'],
      dtype='object')


In [ ]:
data.to_csv('../data/processed/Combined_Flights_2022.csv', index = False)

In [ ]:
data = pd.read_csv('../data/processed/Combined_Flights_2022.csv')
origins = pd.read_csv('../data/processed/origins.csv')
destinations = pd.read_csv('../data/processed/destinations.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4078318 entries, 0 to 4078317
Data columns (total 34 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   Unnamed: 0                               int64  
 1   FlightDate                               object 
 2   Airline                                  object 
 3   Cancelled                                bool   
 4   Diverted                                 bool   
 5   DepTime                                  object 
 6   DepDelayMinutes                          float64
 7   DepDelay                                 float64
 8   ArrTime                                  object 
 9   ArrDelayMinutes                          float64
 10  AirTime                                  float64
 11  CRSElapsedTime                           float64
 12  Distance                                 float64
 13  Year                                     int64  
 14  Quarter           

In [ ]:
data.columns

Index(['Unnamed: 0', 'FlightDate', 'Airline', 'Cancelled', 'Diverted',
       'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime', 'ArrDelayMinutes',
       'AirTime', 'CRSElapsedTime', 'Distance', 'Year', 'Quarter', 'Month',
       'DayofMonth', 'DayOfWeek', 'Marketing_Airline_Network',
       'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline',
       'IATA_Code_Marketing_Airline', 'Flight_Number_Marketing_Airline',
       'Operating_Airline', 'DOT_ID_Operating_Airline',
       'IATA_Code_Operating_Airline', 'Tail_Number',
       'Flight_Number_Operating_Airline', 'OriginAirportID', 'DestAirportID',
       'WheelsOff', 'WheelsOn', 'ArrDelay', 'DivAirportLandings'],
      dtype='object')

In [ ]:
data.head()

In [ ]:
origins.head()

In [ ]:
destinations.head()

From a bussines perspective point of view I wouldn't impute null values, since, for example, DepTime might be null because the flight might have been cancelled and if we impute or drop that registry we might affect the future predictions or even the hypothesis

### Data Acquisition - Database Creation

```
# load the .env file variables
load_dotenv()

# 1) Connect to the database here using the SQLAlchemy's create_engine function
def conn_string():
    user = os.getenv('DB_USER')
    pwd = os.getenv('DB_PASSWORD')
    host = os.getenv('DB_HOST')
    db = os.getenv('DB_NAME')

    return f"postgresql://{user}:{pwd}@{host}/{db}"

def connect():
    global engine
    print("Starting connection...")
    engine = create_engine(conn_string()).execution_options(autocommit=True)
    engine.connect()

    return engine

connect()


########################################################################
# Define your dataset columns and their corresponding data types
columns = {
    'FlightDate': 'DATE',
    'Airline': 'VARCHAR(255)',
    'Origin': 'VARCHAR(255)',
    # Define other columns and their data types here
}

# Generate SQL statement to create table
create_table_sql = f"""
CREATE TABLE flights (
    {", ".join([f"{col} {dtype}" for col, dtype in columns.items()])}
);
"""

# Execute SQL statement to create table
engine.execute(create_table_sql)

# Now, let's say you have your data stored in a pandas DataFrame called 'df'
# You can iterate over the DataFrame rows and generate SQL statements to insert data
for index, row in df.iterrows():
    insert_sql = f"""
    INSERT INTO flights ({", ".join(columns.keys())}) VALUES 
    ({", ".join([str(row[col]) for col in columns.keys()])});
    """
    # Execute SQL statement to insert data
    engine.execute(insert_sql)

########################################################################

# 4) Use pandas to print one of the tables as dataframes using read_sql function
df = pd.read_sql('SELECT * FROM authors', engine)
print(df)
```

In [ ]:
'''load_dotenv()

# 1) Connect to the database here using the SQLAlchemy's create_engine function
def conn_string():
    user = os.getenv('DB_USER')
    pwd = os.getenv('DB_PASSWORD')
    host = os.getenv('DB_HOST')
    db = os.getenv('DB_NAME')

    return f"postgresql://{user}:{pwd}@{host}/{db}"

def connect():
    global engine
    print("Starting connection...")
    engine = create_engine(conn_string()).execution_options(autocommit=True)
    engine.connect()

    return engine

connect()'''

### Data Acquisition - Queries to obtain the data